# T-test

In [72]:
import numpy as np
from scipy.stats import t
from scipy.stats import ttest_ind
import scipy.stats as sps
import plotly.graph_objs as go

In [16]:
size = 1000
equal_var = False
alternatives = ['two-sided', 'less', 'greater']
sample1 = sps.norm.rvs(loc=1, scale=11.5, size=size)
sample2 = sps.norm.rvs(loc=1, scale=1, size=size)
sample3 = sps.norm.rvs(loc=10, scale=1, size=size)

In [17]:
def is_rejected(p_value, alpha=0.05):
    if p_value < alpha:
        return True
    return False


def independent_ttest_unknown_variance(sample1, sample2, alternative='two-sided'):
    n1 = len(sample1)
    n2 = len(sample2)
    dof = n1 + n2 - 2

    mean1 = np.mean(sample1)
    mean2 = np.mean(sample2)
    var1 = np.var(sample1, ddof=1)
    var2 = np.var(sample2, ddof=1)

    sp = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / dof)
    t_stat = (mean1 - mean2) / (sp * np.sqrt(1 / n1 + 1 / n2))

    if alternative == 'two-sided':
        p_value = 2 * (1 - t.cdf(np.abs(t_stat), dof))
    elif alternative == 'less':
        p_value = t.cdf(t_stat, dof)
    elif alternative == 'greater':
        p_value = 1 - t.cdf(t_stat, dof)
    else:
        raise ValueError("Invalid alternative hypothesis. Choose 'two-sided', 'less', or 'greater'.")

    alpha = 0.05
    rejection = is_rejected(p_value, alpha)

    return t_stat, p_value, rejection

In [19]:
for alternative in alternatives:
    print('#'*100)
    print('alternative: {alternative}')
    for sample_1, sample_2, means in [[sample1, sample2, 'same'], [sample2, sample3, 'different']]:
        print('-'*100)
        print(f"means: {means}")
        t_stat, p_value, rejection = independent_ttest_unknown_variance(sample_1, sample_2, alternative)

        print("Custom implementation:")
        print("Test statistic:", t_stat)
        print("p-value:", p_value)
        print("Null hypothesis rejected:", rejection)

        t_stat_scipy, p_value_scipy = ttest_ind(sample_1, sample_2, alternative=alternative, equal_var=equal_var)

        print("\nScipy implementation:")
        print("Test statistic (Scipy):", t_stat_scipy)
        print("p-value (Scipy):", p_value_scipy)
        print("Null hypothesis rejected:", is_rejected(p_value_scipy))

####################################################################################################
alternative: {alternative}
----------------------------------------------------------------------------------------------------
means: same
Custom implementation:
Test statistic: 0.9887093834866207
p-value: 0.3229250671023596
Null hypothesis rejected: False

Scipy implementation:
Test statistic (Scipy): 0.9887093834866207
p-value (Scipy): 0.32304074187708476
Null hypothesis rejected: False
----------------------------------------------------------------------------------------------------
means: different
Custom implementation:
Test statistic: -193.5608016958271
p-value: 0.0
Null hypothesis rejected: True

Scipy implementation:
Test statistic (Scipy): -193.5608016958272
p-value (Scipy): 0.0
Null hypothesis rejected: True
####################################################################################################
alternative: {alternative}
----------------------------------------

# BootStrap

In [23]:
import numpy as np
from scipy.stats import norm, expon

np.random.seed(42)

In [24]:
def bootstrap_confidence_interval(data, estimator, alpha=0.05, n_bootstrap=1000):
    n = len(data)
    estimates = np.zeros(n_bootstrap)
    
    for i in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        estimates[i] = estimator(bootstrap_sample)
    
    lower_bound = np.percentile(estimates, 100 * alpha / 2)
    upper_bound = np.percentile(estimates, 100 * (1 - alpha / 2))
    
    return lower_bound, upper_bound

In [25]:
normal_data = np.random.normal(loc=10, scale=2, size=1000)

median_ci_normal = bootstrap_confidence_interval(normal_data, np.median)
mean_ci_normal = bootstrap_confidence_interval(normal_data, np.mean)

print("Normal Distribution:")
print("Median Confidence Interval:", median_ci_normal)
print("Mean Confidence Interval:", mean_ci_normal)
print()

exponential_data = np.random.exponential(scale=2, size=1000)

median_ci_exponential = bootstrap_confidence_interval(exponential_data, np.median)
mean_ci_exponential = bootstrap_confidence_interval(exponential_data, np.mean)

print("Exponential Distribution:")
print("Median Confidence Interval:", median_ci_exponential)
print("Mean Confidence Interval:", mean_ci_exponential)
print()

mixture_data = np.concatenate([np.random.normal(loc=-5, scale=2, size=500),
                               np.random.normal(loc=5, scale=2, size=500)])

median_ci_mixture = bootstrap_confidence_interval(mixture_data, np.median)
mean_ci_mixture = bootstrap_confidence_interval(mixture_data, np.mean)

print("Mixture of Normal Distributions:")
print("Median Confidence Interval:", median_ci_mixture)
print("Mean Confidence Interval:", mean_ci_mixture)

Normal Distribution:
Median Confidence Interval: (9.904940949335169, 10.193991929985437)
Mean Confidence Interval: (9.911104877808784, 10.16135034629456)

Exponential Distribution:
Median Confidence Interval: (1.250046062603521, 1.4527791204148701)
Mean Confidence Interval: (1.84870193536025, 2.0843084879230793)

Mixture of Normal Distributions:
Median Confidence Interval: (-1.9662048576374929, 1.989938272603765)
Mean Confidence Interval: (-0.36941762097800923, 0.28598959576945326)


# Power and correctness of a test

In [91]:
from scipy.stats import ttest_ind, mannwhitneyu

size = 100
n_exp = 10000
alpha = 0.05

def power_of_a_test(dist, params_1, params_2):
    p_values_ttest = []
    p_values_mannwhitneyu = []
    for i in range(n_exp):
        x_a = dist(**params_1)
        x_b = dist(**params_2)
        p_values_ttest.append(sps.ttest_ind(x_a, x_b, equal_var=False).pvalue)
        p_values_mannwhitneyu.append(sps.mannwhitneyu(x_a, x_b).pvalue)
    p_values_ttest = np.array(p_values_ttest)
    p_values_mannwhitneyu = np.array(p_values_mannwhitneyu)
    print(f'ttest power: {p_values_ttest[p_values_ttest < alpha].shape[0] / p_values_ttest.shape[0] * 100}')
    print(f'mannwhitneyu power: {p_values_mannwhitneyu[p_values_mannwhitneyu < alpha].shape[0] / p_values_mannwhitneyu.shape[0] * 100}')

In [92]:
print('Normal Distribution small effect(1%)')
power_of_a_test(np.random.normal, dict(loc=10, scale=2, size=size), dict(loc=10 + 0.1, scale=2, size=size))

Normal Distribution small effect(1%)
ttest power: 6.41
mannwhitneyu power: 5.94


In [93]:
print('Normal Distribution large effect(10%)')
power_of_a_test(np.random.normal, dict(loc=10, scale=2, size=size), dict(loc=10 + 1, scale=2, size=size))

Normal Distribution large effect(10%)
ttest power: 94.23
mannwhitneyu power: 93.42


In [94]:
print('Exponential Distribution small effect(1%)')
power_of_a_test(np.random.exponential, dict(scale=2, size=size), dict(scale=2 + 0.02, size=size))

Exponential Distribution small effect(1%)
ttest power: 4.859999999999999
mannwhitneyu power: 4.5


In [95]:
print('Exponential Distribution large effect(10%)')
power_of_a_test(np.random.exponential, dict(scale=2, size=size), dict(scale=2 + 0.2, size=size))

Exponential Distribution large effect(10%)
ttest power: 10.52
mannwhitneyu power: 9.1


In [96]:
print('Normal Distribution same means')
power_of_a_test(np.random.normal, dict(loc=10, scale=2, size=size), dict(loc=10, scale=20, size=size))

Normal Distribution same means
ttest power: 4.6
mannwhitneyu power: 9.28


In [97]:
p_values = []
size = size
n_exp = 1000
p_values_ttest = []
p_values_mannwhitneyu = []
for i in range(n_exp):
    x_a = sps.norm.rvs(loc=1, scale=1, size=size)
    x_b = sps.norm.rvs(loc=1, scale=1, size=size)
    p_values_ttest.append(sps.ttest_ind(x_a, x_b, equal_var=False).pvalue)
    p_values_mannwhitneyu.append(sps.mannwhitneyu(x_a, x_b).pvalue)
p_values_ttest = np.array(p_values_ttest)
p_values_mannwhitneyu = np.array(p_values_mannwhitneyu)

In [107]:
probs_ttest = []
probs_mannwhitneyu = []
x = [0.01 * i for i in range(101)]
for i in range(101):
    alpha_step = 0.01 * i
    probs_ttest.append(p_values_ttest[p_values_ttest < alpha_step].shape[0] / p_values_ttest.shape[0])
    probs_mannwhitneyu.append(p_values_mannwhitneyu[p_values_mannwhitneyu < alpha_step].shape[0] / p_values_mannwhitneyu.shape[0])

In [108]:
fig = go.Figure([go.Scatter(x=x, y=probs_ttest, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot ttest")

In [110]:
fig = go.Figure([go.Scatter(x=x, y=probs_mannwhitneyu, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot mannwhitneyu")